In [1]:
import os
import re
import pandas as pd
import numpy as np
from dplython import (DplyFrame, X, diamonds, select, sift, sample_n, sample_frac, arrange, mutate, group_by, summarize)

import plotly.plotly
import plotly.graph_objs as go
import pandas as pd

import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
from plotly import tools

from util_functions import *

%config IPCompleter.greedy=True

in_data_path = '/Users/genie/data/oflc/kaggle/h1b_data_fy2011_fy2018_20190309.csv'
cities_data_path = '/Users/genie/data/oflc/us_cities.csv'
# cities_data_path = '/Users/genie/data/oflc/uscitiesv1.3.csv'
metro_data_path = '/Users/genie/data/oflc/metro_areas_by_county.csv'

coli_data_path = '/Users/genie/data/numbeo/numbeo_coli_2010-2018.csv'

data_zhvi_allhomes = '/Users/genie/data/zillow/City_Zhvi_AllHomes.csv'
data_1bd_rent = '/Users/genie/data/zillow/City_MedianRentalPrice_1Bedroom.csv'

out_dir = '/Users/genie/dev/projects/github/h1b_data_analysis/out'
# out_dir = '/Users/genie/dev/projects/github/genielab_github_io/assets/custom/h1b_data_analysis/'
os.chdir(out_dir)

include_plotlyjs=False

/anaconda3/envs/dev2/lib/python3.7/site-packages/socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
/anaconda3/envs/dev2/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__



In [ ]:
df = pd.read_csv(in_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")
cities_df = pd.read_csv(cities_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")
metro_df = pd.read_csv(metro_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")

coli_df = pd.read_csv(coli_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")

def derive_city_state_in_coli_df(x):
    parts = x['city'].split(',')
    return (str(parts[0].strip()) + ',' + str(parts[1].strip()))

coli_df = coli_df[(coli_df.year==2018) & (coli_df.city.str.contains("United States"))]
coli_df['city_state'] = coli_df.apply(derive_city_state_in_coli_df, axis=1)
coli_df = coli_df.drop(['city','year'],axis=1)

df_zhvi_allhomes = pd.read_csv(data_zhvi_allhomes,na_values='',encoding = "ISO-8859-1")
df_zhvi_allhomes['city_state'] = df_zhvi_allhomes.apply(lambda x: x['RegionName'].strip() + ',' + x['State'].strip(), axis=1)
df_zhvi_allhomes = df_zhvi_allhomes[['city_state','2018-12']]
df_zhvi_allhomes.rename(columns={'2018-12':'zhvi_allhomes_2018'}, inplace=True)

df_1bd_rent = pd.read_csv(data_1bd_rent,na_values='',encoding = "ISO-8859-1")
df_1bd_rent['city_state'] = df_1bd_rent.apply(lambda x: x['RegionName'].strip() + ',' + x['State'].strip(), axis=1)
df_1bd_rent = df_1bd_rent[['city_state','2018-12']]
df_1bd_rent.rename(columns={'2018-12':'rent_1bd_2018'}, inplace=True)

df_zillow = pd.merge(df_zhvi_allhomes, df_1bd_rent, how='left', on=['city_state'])

del df_zhvi_allhomes
del data_1bd_rent

/anaconda3/envs/dev2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
df = df.drop(['case_submitted','decision_date','full_time_position'],axis=1)

# we need to derive all these variables again as they are not accurate, should be dropped in next kaggle dataset version
df = df.drop(['county_fips','county_name','metro','lat','lng'], axis=1)

df['state'] = df.apply(lambda x: x['work_state'] if pd.notnull(x['work_state']) else x['emp_state'] ,axis=1)
df['city'] = df.apply(derive_city_variable, axis=1)

df.city[(pd.notnull(df.city)) & (df.city.str.match("Newyork")==True)] = "New York"
df.city[(pd.notnull(df.city)) & (df.city.str.match("New York City")==True)] = "New York"
df.city[(pd.notnull(df.city)) & (df.city.str.match("Santa Clara")==True)] = "Santa Clara"
df.city[(pd.notnull(df.city)) & (df.city.str.match("San Fransisco")==True)] = "San Francisco"

df['city_state'] = df.apply(lambda x: ','.join([x['city'].title(),x['state']]) if pd.notnull(x['city']) and pd.notnull(x['state']) else None,axis=1)

df = df.drop(['emp_city','emp_state','emp_zip','work_city','work_state','work_zip'], axis=1)

df = pd.merge(df, cities_df[['city_name','state_id','county_fips','county_name']], how='left', left_on=['city','state'], right_on=['city_name','state_id'])
df = df.drop(['city_name','state_id'],axis=1)

df = pd.merge(df, metro_df[['county_fips','metro_name']], how='left', left_on='county_fips', right_on='county_fips')
df.rename(columns={'metro_name':'metro'}, inplace=True)


### Cost of Living 

In [ ]:
df_certified_2018 = df[(df.fiscal_year==2018) & (df.case_status=="C") & (df.city_state.isin(list(coli_df['city_state'].unique())))]
df_certified_2018['avg_annual_wage'] = df_certified_2018.apply(calc_emp_avg_annual_wage, axis=1)

df_certified_2018 = df_certified_2018[['city_state','avg_annual_wage']]
df_coli = pd.merge(df_certified_2018,coli_df,how='left',on=['city_state'])
del df_certified_2018

In [ ]:
# chart
# cost of living analysis

df_coli_1 = DplyFrame(df_coli) >> group_by(X.cost_of_living_index) >> summarize(median_wage=X.avg_annual_wage.median())
df_coli_2 = DplyFrame(df_coli) >> group_by(X.rent_index) >> summarize(median_wage=X.avg_annual_wage.median())
df_coli_3 = DplyFrame(df_coli) >> group_by(X.local_purchasing_power_index) >> summarize(median_wage=X.avg_annual_wage.median())

trace1 = go.Scatter(
    x = list(df_coli_1['cost_of_living_index'].values),
    y = list(df_coli_1['median_wage'].values),
    name = 'Cost of Living Index'
)

trace2 = go.Scatter(
    x = list(df_coli_2['rent_index'].values),
    y = list(df_coli_2['median_wage'].values),
    name = 'Rent Index'
)

trace3 = go.Scatter(
    x = list(df_coli_3['local_purchasing_power_index'].values),
    y = list(df_coli_3['median_wage'].values),
    name = 'Local Purchasing Power Index'
)

data = [trace1, trace2, trace3]

layout = go.Layout(title='Cost of Living Index (FY-2018)',yaxis=dict(title='Median Wage'), legend=dict(x=0,y=1,traceorder='normal',orientation="h"))

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='coli_2018.html')

del df_coli_1
del df_coli_2
del df_coli_3

### House Affordability in Selected Cities

In [ ]:
selected_metros = ['Seattle', 'SF Bay Area', 'Dallas', 'Miami','Atlanta', 'Detroit-Ann Arbor', 'Boston', 'Los Angeles', 'New York-Newark',
       'Charlotte', 'Chicago', 'Tampa', 'Raleigh-Durham', 'Austin', 'Pittsburgh', 'San Diego','Washington DC', 'Philadelphia', 'Milwaukee', 
       'Phoenix', 'Houston','Denver', 'Minneapolis-St.Paul', 'St. Louis', 'Nashville', 'Orlando', 'Kansas City', 'Las Vegas','Sacremento', 
        'Richmond', 'Portland', 'Tulsa', 'Anchorage', 'Honolulu']
selected_metros = sorted(selected_metros)

In [ ]:
df_certified_2018 = df[(df.fiscal_year==2018) & (df.case_status=="C") & (pd.notnull(df.metro)) & (df.metro.isin(selected_metros))]
df_certified_2018['avg_annual_wage'] = df_certified_2018.apply(calc_emp_avg_annual_wage, axis=1)

df_certified_2018 = pd.merge(df_certified_2018,df_zillow,how='left',on=['city_state'])

df_certified_2018['monthly_mortgage_payment'] = df_certified_2018.apply(calc_monthly_mortgage_payment_for_30yrfixed, axis=1)
df_certified_2018['mortgage_as_percent_of_income'] = df_certified_2018.apply(lambda x: round(((x['monthly_mortgage_payment'])/(x['avg_annual_wage']/12))*100,2),axis=1)
df_certified_2018['rent_as_percent_of_income'] = df_certified_2018.apply(lambda x: round(((x['rent_1bd_2018'])/(x['avg_annual_wage']/12))*100,2),axis=1)
df_certified_2018['house_affordability_index'] = df_certified_2018.apply(lambda x: round(100/x['mortgage_as_percent_of_income'],2), axis=1)
df_certified_2018['rent_affordability_index'] = df_certified_2018.apply(lambda x: round(100/x['rent_as_percent_of_income'],2), axis=1)


In [ ]:
# chart
# house affordability index in selected cities (including all percentile groups)

N = len(selected_metros)
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
data = [{
    'y': list(df_certified_2018[(df_certified_2018.metro==selected_metros[i]) & (df_certified_2018.house_affordability_index<=30)]['house_affordability_index'].values), 
    'type':'box',
    'marker':{'color': c[i]},
    'name' : selected_metros[i]
    } for i in range(int(N))]
layout = {'xaxis': {'showgrid':False,'zeroline':False, 'tickangle':90,'showticklabels':True},
          'yaxis': {'zeroline':False,'gridcolor':'white'},
          'paper_bgcolor': 'rgb(233,233,233)',
          'plot_bgcolor': 'rgb(233,233,233)',
          'title': 'House Affordability in Selected Cities (FY-2018)'
          }

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='house_affordability_in_selected_cities_2018.html')

In [ ]:
# chart
# P50-house affordability and P50-rent affordability in selected cities 

x_lables=selected_metros
y_labels=['P50-House Affordability Index','P50-Rent Affordability Index (1bd)']

x_metro_data=[]
for x_metro in selected_metros:
    x_metro_values_1 = df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.house_affordability_index<=30)]['house_affordability_index'].values
    q50_1 = np.nanpercentile(x_metro_values_1,50)
    
    x_metro_values_2 = df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.rent_affordability_index<=30)]['rent_affordability_index'].values
    q50_2 = np.nanpercentile(x_metro_values_2,50)
    x_metro_data.append([round(q50_1,2), round(q50_2,2)])

traces = []
for ylabel_idx,y_label in enumerate(y_labels):
    ylabel_data_points = [x_metro_data[metro_idx][ylabel_idx] for metro_idx in range(len(x_metro_data))]
    traces.append(go.Bar(x=selected_metros,y=ylabel_data_points,name=y_labels[ylabel_idx]))
    
data = [traces[i] for i in range(len(traces))]

layout = go.Layout(barmode='group', title='P50-Housing Affordability Index in Selected Cities (FY-2018)',yaxis=dict(title='P50 Affordability Index'), 
                   legend=dict(x=0,y=1,traceorder='normal',orientation="h"))

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='P50_house_affordability_in_selected_cities_2018.html')

### House Affordability in Selected Employers

In [ ]:
# employer analysis

# df1 = df[(df.case_status=="C") & (df.emp_name.notnull()==True)]
df1 = df[df.emp_name.notnull()==True]
df1['emp_name_x'] = df1['emp_name']

#employer data cleansing
df1.emp_name_x[df1.emp_name.str.match('INFOSYS')] = 'INFOSYS'
df1.emp_name_x[df1.emp_name.str.match('WIPRO')] = 'WIPRO'
df1.emp_name_x[df1.emp_name.str.match('TATA')] = 'TATA (TCS)'
df1.emp_name_x[df1.emp_name.str.match('COGNIZANT')] = 'COGNIZANT'
df1.emp_name_x[df1.emp_name.str.match('CAPGEMINI')] = 'CAPGEMINI'
df1.emp_name_x[df1.emp_name.str.match('DELOITTE')] = 'DELOITTE'
df1.emp_name_x[df1.emp_name.str.match('TECH MAHINDRA')] = 'TECH MAHINDRA'
df1.emp_name_x[df1.emp_name.str.match('IBM')]='IBM'
df1.emp_name_x[df1.emp_name.str.match('L&T|(?=.*LARSEN)(?=.*TOUBRO)')] = 'L&T'
df1.emp_name_x[df1.emp_name.str.match('ACCENTURE')] = 'ACCENTURE'
df1.emp_name_x[df1.emp_name.str.match('HCL')] = 'HCL'
df1.emp_name_x[df1.emp_name.str.match('IGATE')] = 'IGATE'
df1.emp_name_x[df1.emp_name.str.match('UST GLOBAL')] = 'UST GLOBAL'
df1.emp_name_x[df1.emp_name.str.match('(?=.*SYNTEL)(?!.*SYNTELLI|SYNTELLIA)')] = 'SYNTEL'

df1.emp_name_x[df1.emp_name.str.match('PRICEWATERHOUSECOOPERS')] = 'PWC'
df1.emp_name_x[df1.emp_name.str.match('(?=.*ERNST)(?=.*YOUNG)')] = 'ERNST & YOUNG'
df1.emp_name_x[df1.emp_name.str.match('VISA U.S.A.')] = 'VISA'
df1.emp_name_x[df1.emp_name.str.match('CITIBANK|CITIGROUP|CITICORP|CITIFINANCIAL|CITIMORTGAGE|CITI VENTURES|CITI HEDGE FUND')] = 'CITIBANK'
df1.emp_name_x[df1.emp_name.str.match('DISCOVER PRODUCTS|DISCOVER GROUP|DISCOVER BANK')] = 'DISCOVER'
df1.emp_name_x[df1.emp_name.str.match('JPMORGAN')] = 'JPMORGAN CHASE'
df1.emp_name_x[df1.emp_name.str.match('(?=.*AMERICAN EXPRESS)(?!.*LOGISTICS)')] = 'AMERICAN EXPRESS'
df1.emp_name_x[df1.emp_name.str.match('WELLS FARGO')] = 'WELLS FARGO'
df1.emp_name_x[df1.emp_name.str.match('BANK OF AMERICA')] = 'BANK OF AMERICA'
df1.emp_name_x[df1.emp_name.str.match('(?=.*CAPITAL ONE)(?!.*CONSTRUCTION)')] = 'CAPITAL ONE'
df1.emp_name_x[df1.emp_name.str.match('GOLDMAN SACH')] = 'GOLDMAN SACH'
df1.emp_name_x[df1.emp_name.str.match('MORGAN STANLEY')] = 'MORGAN STANLEY'

df1.emp_name_x[df1.emp_name.str.match('AMAZON')] = 'AMAZON'
df1.emp_name_x[df1.emp_name.str.match('GOOGLE')] = 'GOOGLE'
df1.emp_name_x[df1.emp_name.str.match('FACEBOOK')] = 'FACEBOOK'
df1.emp_name_x[df1.emp_name.str.match('APPLE INC')] = 'APPLE'
df1.emp_name_x[df1.emp_name.str.match('TESLA')] = 'TESLA'
df1.emp_name_x[df1.emp_name.str.match('MICROSOFT')] = 'MICROSOFT'
df1.emp_name_x[df1.emp_name.str.match('YAHOO')] = 'YAHOO'
df1.emp_name_x[df1.emp_name.str.match('ORACLE AMERICA|ORACLE FINANCIAL|ORACLE USA')] = 'ORACLE'
df1.emp_name_x[df1.emp_name.str.match('NETFLIX')] = 'NETFLIX'
df1.emp_name_x[df1.emp_name.str.match('UBERCAB|UBER, INC|UBER TECHNOLOGIES')] = 'UBER'
df1.emp_name_x[df1.emp_name.str.match('AIRBNB')] = 'AIRBNB'
df1.emp_name_x[df1.emp_name.str.match('QUALCOMM')] = 'QUALCOMM'
df1.emp_name_x[df1.emp_name.str.match('INTEL CORPORATION|INTEL AMERICA|INTEL INC')] = 'INTEL'
df1.emp_name_x[df1.emp_name.str.match('CISCO SYSTEMS|CISCO SYSTEM|CISCO CONSUMER|CISCO FC')] = 'CISCO'

df1.emp_name_x[df1.emp_name.str.match('WAL-MART')] = 'WALMART'
df1.emp_name_x[df1.emp_name.str.match('COSTCO')] = 'COSTCO'
df1.emp_name_x[df1.emp_name.str.match('HOME DEPOT')] = 'HOME DEPOT'
df1.emp_name_x[df1.emp_name.str.match('MACYS')] = 'MACYS'
df1.emp_name_x[df1.emp_name.str.match('VERIZON')] = 'VERIZON'
df1.emp_name_x[df1.emp_name.str.match('AT&T')] = 'AT&T'

df1.emp_name_x[df1.emp_name.str.match('CHEVRON')] = 'CHEVRON'
df1.emp_name_x[df1.emp_name.str.match('EXXON MOBIL|EXXONMOBIL')] = 'EXXON MOBIL'
df1.emp_name_x[df1.emp_name.str.match('BP AMERICA')] = 'BRITISH PETROLEUM (BP)'
df1.emp_name_x[df1.emp_name.str.match('SHELL EXPLORATION|SHELL INTERNATIONAL|SHELL GLOBAL|SHELL OIL')] = 'SHELL OIL'

df1.emp_name_x[df1.emp_name.str.match('MASTECH DIGITAL')] = 'MASTECH'
df1.emp_name_x[df1.emp_name.str.match('NTT DATA')] = 'NTT DATA'
df1.emp_name_x[df1.emp_name.str.match('SALESFORCE.COM')] = 'SALESFORCE'
df1.emp_name_x[df1.emp_name.str.match('LINKEDIN')] = 'LINKEDIN'
df1.emp_name_x[df1.emp_name.str.match('VIRTUSA')] = 'VIRTUSA'
df1.emp_name_x[df1.emp_name.str.match('VMWARE')] = 'VMWARE'
df1.emp_name_x[df1.emp_name.str.match('PAYPAL')] = 'PAYPAL'
df1.emp_name_x[df1.emp_name.str.match('EBAY')] = 'EBAY'
df1.emp_name_x[df1.emp_name.str.match('HEXAWARE')] = 'HEXAWARE'
df1.emp_name_x[df1.emp_name.str.match('FUJITSU AMERICA')] = 'FUJITSU AMERICA'
df1.emp_name_x[df1.emp_name.str.match('EMC CORPORATION')] = 'EMC'
df1.emp_name_x[df1.emp_name.str.match('SATYAM COMPUTER SERVICES')] = 'SATYAM COMPUTER SERVICES'
df1.emp_name_x[df1.emp_name.str.match('PATNI AMERICAS')] = 'PATNI'

df1['avg_annual_wage'] = df1.apply(calc_emp_avg_annual_wage, axis=1)

In [ ]:
selected_companies = ['INFOSYS','WIPRO','TATA (TCS)','COGNIZANT','CAPGEMINI','DELOITTE','IBM',
                      'CITIBANK','AMERICAN EXPRESS','BANK OF AMERICA','CAPITAL ONE','JPMORGAN CHASE','WELLS FARGO',
                      'AMAZON','GOOGLE','MICROSOFT','ORACLE','APPLE','TESLA','NETFLIX','UBER','FACEBOOK', 'AIRBNB','INTEL','CISCO',
                      'CHEVRON','EXXON MOBIL','SHELL OIL','BRITISH PETROLEUM (BP)','WALMART','COSTCO','VERIZON','AT&T'
                     ]

selected_companies = sorted(selected_companies)

In [ ]:
oflc_2018_selected_companies = df1[(df1.fiscal_year==2018) & (df1.case_status=="C") & (df1.emp_name_x.isin(selected_companies))]
oflc_2018_selected_companies['avg_annual_wage'] = oflc_2018_selected_companies.apply(calc_emp_avg_annual_wage, axis=1)

oflc_2018_selected_companies = pd.merge(oflc_2018_selected_companies,df_zillow,how='left',on=['city_state'])

oflc_2018_selected_companies['monthly_mortgage_payment'] = oflc_2018_selected_companies.apply(calc_monthly_mortgage_payment_for_30yrfixed, axis=1)
oflc_2018_selected_companies['mortgage_as_percent_of_income'] = oflc_2018_selected_companies.apply(lambda x: round(((x['monthly_mortgage_payment'])/(x['avg_annual_wage']/12))*100,2),axis=1)
oflc_2018_selected_companies['rent_as_percent_of_income'] = oflc_2018_selected_companies.apply(lambda x: round(((x['rent_1bd_2018'])/(x['avg_annual_wage']/12))*100,2),axis=1)
oflc_2018_selected_companies['house_affordability_index'] = oflc_2018_selected_companies.apply(lambda x: round(100/x['mortgage_as_percent_of_income'],2), axis=1)
oflc_2018_selected_companies['rent_affordability_index'] = oflc_2018_selected_companies.apply(lambda x: round(100/x['rent_as_percent_of_income'],2), axis=1)


In [ ]:
# chart
# house affordability index in selected companies (including all percentile groups)

N = len(selected_companies)
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
data = [{
    'y': list(oflc_2018_selected_companies[(oflc_2018_selected_companies.emp_name_x==selected_companies[i]) & (oflc_2018_selected_companies.house_affordability_index<=30)]['house_affordability_index'].values), 
    'type':'box',
    'marker':{'color': c[i]},
    'name' : selected_companies[i]
    } for i in range(int(N))]
layout = {'xaxis': {'showgrid':False,'zeroline':False, 'tickangle':90,'showticklabels':True},
          'yaxis': {'zeroline':False,'gridcolor':'white'},
          'paper_bgcolor': 'rgb(233,233,233)',
          'plot_bgcolor': 'rgb(233,233,233)',
          'title': 'House Affordability in Selected Companies (FY-2018)'
          }

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='house_affordability_in_selected_companies_2018.html')

In [ ]:
# chart
# P50-house affordability and P50-rent affordability in selected companies 

x_lables=selected_companies
y_labels=['P50-House Affordability Index','P50-Rent Affordability Index (1bd)']

x_emp_data=[]
for x_emp in selected_companies:
    x_emp_values_1 = oflc_2018_selected_companies[(oflc_2018_selected_companies.emp_name_x==x_emp) & (oflc_2018_selected_companies.house_affordability_index<=30)]['house_affordability_index'].values
    q50_1 = np.nanpercentile(x_emp_values_1,50)
    x_emp_values_2 = oflc_2018_selected_companies[(oflc_2018_selected_companies.emp_name_x==x_emp) & (oflc_2018_selected_companies.rent_affordability_index<=30)]['rent_affordability_index'].values
    q50_2 = np.nanpercentile(x_emp_values_2,50)
    x_emp_data.append([round(q50_1,2), round(q50_2,2)])

traces = []
for ylabel_idx,y_label in enumerate(y_labels):
    ylabel_data_points = [x_emp_data[emp_idx][ylabel_idx] for emp_idx in range(len(x_emp_data))]
    traces.append(go.Bar(x=selected_companies,y=ylabel_data_points,name=y_labels[ylabel_idx]))
    
data = [traces[i] for i in range(len(traces))]

layout = go.Layout(barmode='group', title='P50-Housing Affordability in Selected Companies (FY-2018)',
                   yaxis=dict(title='P50-House Affordability Index'), legend=dict(x=0,y=1,traceorder='normal',orientation="h"))

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='P50_house_affordability_in_selected_companies_2018.html')


In [ ]:

# # federal income tax
# df_certified_2018['federal_income_tax'] = df_certified_2018.apply(federal_income_tax_2018,axis=1)

# # defualt state tax rate for tax free states
# df_certified_2018['state_income_tax_rate'] = df_certified_2018.apply(lambda x: 0, axis=1)
# df_certified_2018['state_income_tax'] = df_certified_2018.apply(lambda x: 0, axis=1)

# df_certified_2018.state_income_tax_rate[df_certified_2018.state.isin(['WA','TX','AK','FL','NV'])] = df_certified_2018.apply(lambda x: 0, axis=1)
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MI"] = df_certified_2018.apply(lambda x: 4.25, axis=1)
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MA"] = df_certified_2018.apply(lambda x: 5.10, axis=1)
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="NC"] = df_certified_2018.apply(lambda x: 5.50, axis=1)
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="IL"] = df_certified_2018.apply(lambda x: 4.95, axis=1)
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="PA"] = df_certified_2018.apply(lambda x: 3.07, axis=1)
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="CO"] = df_certified_2018.apply(lambda x: 4.63, axis=1)
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="IN"] = df_certified_2018.apply(lambda x: 3.23, axis=1)
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="TN"] = df_certified_2018.apply(lambda x: 3.00, axis=1)

# # tax brackets for selected states
# # https://taxfoundation.org/
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="CA"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,8223,19495,30769,42711,53980,275738,330884,551473,1000000,np.inf], 
#                                                                                 labels=[1.00,2.00,3.00,4.00,8.00,9.30,10.30,11.30,12.30,13.30])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="GA"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,750,2250,2750,5250,7000,np.inf], 
#                                                                                 labels=[1.00,2.00,3.00,4.00,5.00,6.00])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="AZ"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,10346,25861,51721,155159, np.inf], 
#                                                                      labels=[2.59,2.88,3.36,4.24,4.54])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="NJ"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,20000,35000,40000,75000,500000, np.inf], 
#                                                                      labels=[1.40,1.75,3.50,5.53,6.37,8.97])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="NY"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,8500,11700,13900,21400,80650,215400,1077550, np.inf], 
#                                                                      labels=[4.00,4.50,5.25,5.90,6.33,6.57,6.85,8.82])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MD"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,1000,2000,3000,100000,125000,150000,250000, np.inf], 
#                                                                      labels=[2.00,3.00,4.00,4.75,5.00,5.25,5.50,5.75])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="WI"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,11230,22470,247350, np.inf], 
#                                                                      labels=[4.00,5.84,6.27,7.65])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="VA"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,3000,5000,17000, np.inf], 
#                                                                      labels=[2.00,3.00,5.00,5.75])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MN"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,25890,85060,160020, np.inf], 
#                                                                      labels=[5.35,7.05,7.85,9.85])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="DC"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,10000,40000,60000,350000,1000000, np.inf], 
#                                                                      labels = [4.00,6.00,6.50,8.50,8.75,8.95])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MO"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[100,1008,2016,3024,4032,5040,6048,7056,8064,9072, np.inf], 
#                                                                      labels = [1.50,2.00,2.50,3.00,3.50,4.00,4.50,5.00,5.50,5.90])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="KS"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[2500,15000,30000, np.inf], 
#                                                                      labels = [3.10,5.25,5.70])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="OR"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,3450,8700,125000, np.inf], 
#                                                                      labels = [5.00,7.00,9.00,9.90])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="OK"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,1000,2500,3750,4900,7200, np.inf], 
#                                                                      labels = [0.50,1.00,2.00,3.00,4.00,5.00])
# df_certified_2018.state_income_tax_rate[df_certified_2018.state=="HI"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,2400,4800,9600,14400,19200,24000,36000,48000,150000,175000,200000, np.inf], 
#                                                                      labels = [1.40,3.20,5.50,6.40,6.80,7.20,7.60,7.90,8.25,9.00,10.00,11.00])

# df_certified_2018['state_income_tax'] = df_certified_2018.apply(lambda x: round(x['state_income_tax_rate']*x['avg_annual_wage']/100,2), axis=1)

# # https://www.irs.gov/taxtopics/tc751
# df_certified_2018['social_security_tax'] = df_certified_2018.apply(lambda x: round(6.2*x['avg_annual_wage']/100,2) if x['avg_annual_wage']<128400.00 else round(6.2*128400.00/100,2), axis=1)
# df_certified_2018['medicare_tax'] = df_certified_2018.apply(lambda x: round(1.45*x['avg_annual_wage']/100,2), axis=1)

# df_certified_2018['all_taxes'] = df_certified_2018.apply(lambda x: x['federal_income_tax']+x['state_income_tax']+x['social_security_tax']+x['medicare_tax'], axis=1)
# df_certified_2018['wage_after_taxes'] = df_certified_2018.apply(lambda x: x['avg_annual_wage']-x['all_taxes'], axis=1)
